# Terraform Import Example

## Table of Contents

1. [Overview](#overview)
1. [Requirements](#requirements)
1. [Workflow](#workflow)
    * [Create an orphaned resource](#create-an-orphaned-resource)
    * [Import and orphaned resource](#import-and-orphaned-resource)
    * [Generate Terraform code](#generate-terraform-code)
    * [Run generated code](#run-generated-code)
    * [Destroy the resource](#destroy-the-resource)
1. [Demo Reset](#demo-reset)

## Overview

An example workflow for importing an unmanaged resource into Terraform state and generating the corresponding code.

All content originated from Ranjit's wonderful repo. https://github.com/WhatsARanjit/terraform-import_demo

## Requirements

* `terraform` version >= 0.12.0
* AWS credentials
* `jq`

### Set Main Environment Variables

In [ ]:
export RED="\e[0;31m" YELLOW="\e[0;33m" BLDYELLOW="\e[1;33m" GREEN="\e[0;32m"
export CYAN="\e[0;36m" BLUE="\e[0;34m" WHITE="\e[0;37m" BLDWHITE="\e[1;37m"
export NC="\e[0m"

printf "${GREEN}# Set MAIN_DIR for future reference.${NC}\n"
export MAIN_DIR=${PWD}

Be sure to replace `TF_ORG` and `TF_WS_NAME` with your own values.

In [ ]:
export TF_GIT_DIR="config/terraform/terraform-import_demo"
printf "${GREEN}# Set Terraform Variables.${NC}\n"
printf "\n$MAIN_DIR \n $TF_GIT_DIR \n"

# Some commands may have sensitive information. Prevent commands starting with a space to be saved to shell history.
HISTCONTROL=ignoreboth # do not save lines that begin with space in history

### AWS Credentials

`terraform` will use credentials set in your environment or through other means as described in the [Terraform documentation](https://www.terraform.io/docs/providers/aws/index.html#environment-variables). This guide will assume you are using the "**Environment Variables**" method.

Add your AWS credentials as two environment variables. Set your `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` replacing `AAAAAA` with your own values.

In [ ]:
 export AWS_ACCESS_KEY_ID=AAAAAA
 export AWS_SECRET_ACCESS_KEY=AAAAAA
#  export AWS_SESSION_TOKEN=AAAAAA

NOTE: If you use `doormat`, then use this method.

In [ ]:
alias awscredsenv='doormat --smoke-test || doormat -r && eval $(doormat aws -a $account)'
awscredsenv

### Clone Repo

In [ ]:
git clone https://github.com/WhatsARanjit/terraform-import_demo.git $TF_GIT_DIR

## Workflow

### Create an orphaned resource

Start by creating an example VPC in AWS to play with:

In [ ]:
# pushd examples/
terraform -chdir=$TF_GIT_DIR/example init

In [ ]:
terraform -chdir=$TF_GIT_DIR/example apply -auto-approve

Grab the `VPC_ID` for later:

In [ ]:
export VPC_ID=$(terraform -chdir=${TF_GIT_DIR}/example output -json | jq -r '.ARN.value' | rev | cut -d/ -f1 | rev)
echo $VPC_ID

### Import and orphaned resource

Let's move back to the main directory. We will specify the `root` folder for the next few steps.

* For all intents and purposes, working out of this directory, Terraform has no idea of any state.
* To make a strong point, you can even consider deleting the `terraform.tfstate` and `terraform.tfstate.backup` files to simulate an event.

Attempt to import the VPC with the name `main`:

In [ ]:
export TF_RESOURCE_NAME="main"
terraform -chdir=$TF_GIT_DIR import aws_vpc.$TF_RESOURCE_NAME $VPC_ID

You'll see that Terraform is unhappy because there is no resource in code for `aws_vpc.main`
* This is kind of a chicken & egg problem

Let's stub a resource:

In [ ]:
tee $TF_GIT_DIR/${TF_RESOURCE_NAME}.tf << EOF
resource "aws_vpc" "${TF_RESOURCE_NAME}" {}
EOF

Now we have a stubbed resource

Attempt to import the resource again:

In [ ]:
terraform -chdir=$TF_GIT_DIR init

In [ ]:
terraform -chdir=$TF_GIT_DIR import aws_vpc.$TF_RESOURCE_NAME $VPC_ID

* Now Terraform is unhappy because the provider is not completely configured

We have to supply the region information, which is not actually in the statefile itself:

In [ ]:
cat > $TF_GIT_DIR/versions.tf << EOF
provider "aws" {
  region = "us-east-1"
}
EOF

Now attempt to import the resource:

In [ ]:
terraform -chdir=$TF_GIT_DIR import aws_vpc.$TF_RESOURCE_NAME $VPC_ID

### Generate Terraform code

You can generate Terraform code from state like this. **NOTE:** We're overwriting the stub with real code.

In [ ]:
terraform -chdir=$TF_GIT_DIR show -no-color > $TF_GIT_DIR/$TF_RESOURCE_NAME.tf
cat $TF_GIT_DIR/$TF_RESOURCE_NAME.tf

Running the generated code from state should produce a no-changes apply:

In [ ]:
terraform -chdir=$TF_GIT_DIR apply -auto-approve

**NOTE:** When running on a terminal, Terraform recognizes you are a human and returns HCL.  When running inside of a script Terraform decides to return JSON instead.

### Run generated code

* We see that Terraform is unhappy because the code is specifying values for read-only attrbutes
* Remove the read-only attributes from the code, which are:
  * `arn`
  * `default_network_acl_id`
  * `default_route_table_id`
  * `default_security_group_id`
  * `dhcp_options_id`
  * `id`
  * `main_route_table_id`
  * `owner_id`

In [ ]:
sed -i '.bak' -e '/^ *arn/d' -e '/^ *default_network_acl_id/d' \
  -e '/^ *default_route_table_id/d' -e '/^ *default_security_group_id/d' \
  -e '/^ *dhcp_options_id/d' -e '/^ *id/d' \
  -e '/^ *main_route_table_id/d' -e '/^ *owner_id/d' \
  $TF_GIT_DIR/$TF_RESOURCE_NAME.tf

> **NOTE:** Deleting lines in the above manner could accidentally match unintended lines.

Now try the code again:

In [ ]:
terraform -chdir=$TF_GIT_DIR apply -auto-approve

You should see that there are no changes.

```shell
aws_vpc.main: Refreshing state... [id=vpc-033922d140d7e8a0c]

No changes. Your infrastructure matches the configuration.

Terraform has compared your real infrastructure against your configuration and
found no differences, so no changes are needed.

Apply complete! Resources: 0 added, 0 changed, 0 destroyed.
```

### Destroy the resource

Now that we have the resource under management and corresponding code, we can take it the end of its lifecycle:

In [ ]:
terraform -chdir=$TF_GIT_DIR destroy -auto-approve

## Clean Up

In [ ]:
rm -rf $TF_GIT_DIR
unset AWS_DEFAULT_REGION